In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bredasdorp,ZA,2021-05-25 20:24:56,-34.5322,20.0403,52.92,76,72,6.33
1,1,Padang,ID,2021-05-25 20:24:08,-0.9492,100.3543,78.06,88,91,4.00
2,2,Cape Town,ZA,2021-05-25 20:24:56,-33.9258,18.4232,58.12,79,0,5.08
3,3,Atuona,PF,2021-05-25 20:24:57,-9.8000,-139.0333,79.86,64,40,14.94
4,4,Ushuaia,AR,2021-05-25 20:24:47,-54.8000,-68.3000,31.66,86,90,21.85


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
# alternatively can use list comprehension in weights param in heat_layer
# weights=[max(temp,0) for temp in max_temp]
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300,point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

## problem - settings for dissipating, max_intensity and point_radius disappear when clicking on image to zoom.scroll 
## same problem with humidity

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Padang,ID,2021-05-25 20:24:08,-0.9492,100.3543,78.06,88,91,4.00
3,3,Atuona,PF,2021-05-25 20:24:57,-9.8000,-139.0333,79.86,64,40,14.94
8,8,Puerto Ayora,EC,2021-05-25 20:24:58,-0.7393,-90.3518,85.96,87,39,8.99
10,10,Kudahuvadhoo,MV,2021-05-25 20:24:59,2.6708,72.8944,83.86,74,100,22.44
11,11,Pacific Grove,US,2021-05-25 20:24:59,36.6177,-121.9166,82.15,73,1,10.00


In [12]:
preferred_cities_df.count()

City_ID       175
City          175
Country       175
Date          175
Lat           175
Lng           175
Max Temp      175
Humidity      175
Cloudiness    175
Wind Speed    175
dtype: int64

In [75]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""



In [76]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame. 
# For each preferred city, do a nearby search of hotels and add the first result as the hotel name for the city
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json() 
    
    # Parse each result and test for exceptions
    # Grab the first hotel from the results and store the name.
#hotels["results"][0]["name"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]  
    except(IndexError):
        print("Hotel not found ... skipping")
        pass
    



Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [77]:
print(type(hotel_df))

<class 'pandas.core.frame.DataFrame'>


In [78]:
import numpy as np
clean_hotel_df=hotel_df.replace('',np.nan)
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Padang,ID,78.06,-0.9492,100.3543,Mercure Padang
3,Atuona,PF,79.86,-9.8000,-139.0333,Villa Enata
8,Puerto Ayora,EC,85.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Kudahuvadhoo,MV,83.86,2.6708,72.8944,Niyama Private Islands Maldives
11,Pacific Grove,US,82.15,36.6177,-121.9166,Lovers Point Inn
14,Bonthe,SL,80.78,7.5264,-12.5050,Bonthe Holiday Village
15,Tecoanapa,MX,89.65,16.5167,-98.7500,Hotel Manglares
18,Albany,US,82.62,42.6001,-73.9662,NaN
20,Hilo,US,83.80,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Jamestown,US,85.08,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [80]:
clean_hotel_drop_df=clean_hotel_df.dropna() #didn't work - have NaNs
#clean_hotel_drop_df.isnull().sum() #have 9 NaNs
clean_hotel_drop_df.count() #NaNs not counted but rows not deleted by dropna


City          166
Country       166
Max Temp      166
Lat           166
Lng           166
Hotel Name    166
dtype: int64

In [81]:
clean_hotel_drop_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Padang,ID,78.06,-0.9492,100.3543,Mercure Padang
3,Atuona,PF,79.86,-9.8000,-139.0333,Villa Enata
8,Puerto Ayora,EC,85.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Kudahuvadhoo,MV,83.86,2.6708,72.8944,Niyama Private Islands Maldives
11,Pacific Grove,US,82.15,36.6177,-121.9166,Lovers Point Inn
14,Bonthe,SL,80.78,7.5264,-12.5050,Bonthe Holiday Village
15,Tecoanapa,MX,89.65,16.5167,-98.7500,Hotel Manglares
20,Hilo,US,83.80,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Jamestown,US,85.08,42.0970,-79.2353,Hampton Inn & Suites Jamestown
25,San Andres,CO,87.82,12.5847,-81.7006,Hotel Arena Blanca


In [85]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = clean_hotel_drop_df[["Lat", "Lng"]]
max_temp = clean_hotel_drop_df["Max Temp"]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_drop_df.iterrows()]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))